# 特征工程

本笔记本展示了特征工程的常用方法，包括创建交互特征、多项式特征等，以及它们对模型性能的影响。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. 生成房屋价格预测数据

In [ ]:
def generate_house_data():
    """
    生成房屋价格预测的模拟数据
    
    返回:
    X: 特征数据 (n_samples, 4)，包含：
        - 房屋面积 (平方英尺)
        - 卧室数量
        - 房龄 (年)
        - 距离市中心距离 (英里)
    y: 房屋价格 (美元)
    """
    n_samples = 200
    # 生成特征数据
    size = np.random.rand(n_samples, 1) * 2000  # 房屋面积
    bedrooms = np.random.randint(1, 6, size=(n_samples, 1))  # 卧室数量
    age = np.random.rand(n_samples, 1) * 50  # 房龄
    distance = np.random.rand(n_samples, 1) * 10  # 距离
    # 生成噪声
    noise = np.random.randn(n_samples, 1) * 50000
    
    # 生成价格 (考虑特征之间的交互)
    price = 100000 + 150 * size + 50000 * bedrooms - 1000 * age - 15000 * distance + noise
    
    # 合并特征
    X = np.hstack((size, bedrooms, age, distance))
    return X, price

# 生成数据
X, y = generate_house_data()
print(f"数据形状: X={X.shape}, y={y.shape}")
print("特征包括: 房屋面积, 卧室数量, 房龄, 距离市中心距离")

## 2. 创建交互特征

In [ ]:
def create_interaction_features(X):
    """
    创建交互特征
    
    参数:
    X: 原始特征矩阵 (m, 4)
    
    返回:
    X_with_interactions: 包含交互特征的矩阵 (m, 6)
    """
    size = X[:, 0:1]          # 房屋面积
    bedrooms = X[:, 1:2]      # 卧室数量
    age = X[:, 2:3]           # 房龄
    distance = X[:, 3:4]      # 距离
    
    # 创建交互特征
    size_per_bedroom = size / (bedrooms + 0.1)  # 每卧室面积
    age_distance = age * distance               # 房龄与距离的交互
    
    # 合并特征
    X_with_interactions = np.hstack((X, size_per_bedroom, age_distance))
    return X_with_interactions

# 创建交互特征
X_with_interactions = create_interaction_features(X)
print(f"添加交互特征后形状: {X_with_interactions.shape}")

## 3. 创建多项式特征

In [ ]:
def create_polynomial_features(X, degree=2):
    """
    创建多项式特征
    
    参数:
    X: 原始特征矩阵 (m, n)
    degree: 多项式次数
    
    返回:
    X_poly: 包含多项式特征的矩阵
    """
    n_samples, n_features = X.shape
    X_poly = X.copy()
    
    # 为每个特征创建多项式
    for i in range(n_features):
        for d in range(2, degree + 1):
            # 添加d次方特征
            feature_power = np.power(X[:, i:i+1], d)
            X_poly = np.hstack((X_poly, feature_power))
    
    return X_poly

# 创建多项式特征
X_poly = create_polynomial_features(X, degree=2)
print(f"添加2阶多项式特征后形状: {X_poly.shape}")

## 4. 组合所有特征工程方法

In [ ]:
# 先创建多项式特征，再创建交互特征
X_combined = create_interaction_features(X_poly)
print(f"组合所有特征后形状: {X_combined.shape}")

## 5. 梯度下降实现

In [ ]:
def gradient_descent(X, y, theta, learning_rate, n_iterations):
    """
    使用梯度下降算法训练线性回归模型
    
    参数:
    X: 特征矩阵 (m, n+1)，包含偏置项
    y: 目标值向量 (m, 1)
    theta: 初始参数向量 (n+1, 1)
    learning_rate: 学习率
    n_iterations: 迭代次数
    
    返回:
    theta: 学习后的参数向量
    cost_history: 每次迭代的代价函数值
    """
    m = len(y)  # 样本数量
    cost_history = np.zeros(n_iterations)  # 记录代价函数历史
    
    for i in range(n_iterations):
        # 计算预测值
        predictions = X.dot(theta)
        # 计算误差
        errors = predictions - y
        # 计算梯度
        gradients = (1/m) * X.T.dot(errors)
        # 更新参数
        theta = theta - learning_rate * gradients
        # 记录当前代价函数值
        cost_history[i] = (1/(2*m)) * np.sum(np.square(errors))
    
    return theta, cost_history

## 6. 训练不同特征集的模型

In [ ]:
def train_and_evaluate(X_train):
    """
    训练模型并评估性能
    
    参数:
    X_train: 特征矩阵
    
    返回:
    cost_history: 训练过程的代价函数历史
    """
    # 添加偏置项
    X_b = np.c_[np.ones((len(X_train), 1)), X_train]
    # 初始化参数
    theta_initial = np.random.randn(X_b.shape[1], 1)
    # 设置超参数
    learning_rate = 0.00000001
    n_iterations = 1000
    # 训练模型
    theta, cost_history = gradient_descent(X_b, y, theta_initial, learning_rate, n_iterations)
    return cost_history

# 训练不同特征集的模型
cost_original = train_and_evaluate(X)
cost_interactions = train_and_evaluate(X_with_interactions)
cost_poly = train_and_evaluate(X_poly)
cost_combined = train_and_evaluate(X_combined)

## 7. 可视化不同特征工程方法的性能

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(1000), cost_original, label='原始特征')
plt.plot(range(1000), cost_interactions, label='添加交互特征')
plt.plot(range(1000), cost_poly, label='添加多项式特征')
plt.plot(range(1000), cost_combined, label='组合所有特征')
plt.xlabel('迭代次数')
plt.ylabel('代价函数值')
plt.title('不同特征工程方法的性能对比')
plt.legend()
plt.grid(True)
plt.show()

## 8. 模型性能评估

In [ ]:
print("特征工程效果评估:")
print(f"原始特征最终代价: {cost_original[-1]:.4f}")
print(f"交互特征最终代价: {cost_interactions[-1]:.4f}")
print(f"多项式特征最终代价: {cost_poly[-1]:.4f}")
print(f"组合特征最终代价: {cost_combined[-1]:.4f}")

# 计算性能提升百分比
base_cost = cost_original[-1]
print(f"\n性能提升百分比:")
print(f"交互特征: {(base_cost - cost_interactions[-1])/base_cost * 100:.2f}%")
print(f"多项式特征: {(base_cost - cost_poly[-1])/base_cost * 100:.2f}%")
print(f"组合特征: {(base_cost - cost_combined[-1])/base_cost * 100:.2f}%")

## 9. 特征工程总结

In [ ]:
print("特征工程的重要性:")
print("1. 可以捕获特征之间的非线性关系")
print("2. 可以提高模型的预测能力")
print("3. 可以减少模型的复杂度")
print("4. 可以使模型更好地泛化")

print("\n常用的特征工程方法:")
print("- 交互特征: 捕捉特征之间的相互作用")
print("- 多项式特征: 捕捉非线性关系")
print("- 特征缩放: 标准化或归一化特征")
print("- 特征选择: 选择最相关的特征")
print("- 特征提取: 从原始特征中提取新特征")